In [14]:
from counterfactuals.models import Cadex
from counterfactuals.visualization import compare
from data import GermanData
from tensorflow import keras

In [15]:
german_data = GermanData('data/input_german.csv', 'data/labels_german.csv')
model = keras.models.load_model('models/model_german')
test_X = german_data.X_test.iloc[6]
cadex = Cadex(model)
cf = cadex.generate(test_X)

In [16]:
df = compare(test_X, cf)
df

,X,X',change
phone_none,-1.224745,-0.438257,0.786488
account_status_no checking account,-0.778733,0.052315,0.831048
purpose_radio/television,-0.633286,0.197958,0.831244
employment_4..7 years,-0.458571,0.375994,0.834565
purpose_car (used),-0.331014,0.489743,0.820757


In [17]:
test_X = german_data.input.iloc[6]
cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)
cf = cadex.generate(test_X)
compare(test_X, cf)

,X,X',change
account_status_< 0 DM,0,1.008654e+00,1.008654e+00
credit_history_no credits taken,0,4.589541e-01,4.589541e-01
purpose_car (new),0,9.094295e-01,9.094295e-01
job_unskilled - resident,0,8.266369e-01,8.266369e-01
duration,24,5.019575e+01,2.619575e+01
housing_for free,0,-1.387779e-17,-1.387779e-17


In [18]:
test_X = german_data.X_test.iloc[10]
cadex = Cadex(model, n_changed=8)
cf = cadex.generate(test_X)
compare(test_X, cf)

,X,X',change
installment_percent,0.032104,1.281978,1.249874
savings_< 100 DM,-1.267227,0.099308,1.366535
account_status_< 0 DM,-0.615882,0.690943,1.306825
purpose_car (new),-0.565802,0.779822,1.345624
employment_< 1 year,-0.456573,0.712271,1.168844
other_installment_plans_bank,-0.411814,0.778749,1.190563
credit_history_delay in paying off in the past,-0.302333,1.057303,1.359636
credit_history_no credits taken,-0.213896,0.979219,1.193116
